# Duhamel Methods Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lawrennd/qig-code/blob/main/examples/duhamel_methods_comparison.ipynb)

Compare four implementations of Kubo–Mori / Duhamel derivatives for quantum exponential families:

- Quadrature (trapezoid)
- Spectral/BCH
- Block matrix (Higham)
- SLD approximation

Links:
- API docs: `docs/source/api/duhamel.rst`
- Comparison guide: `docs/duhamel_methods_comparison.md`
- CIP: `cip/cip000A.md`


## Environment setup

Run this cell on Colab to install dependencies and set plotting style.


In [ ]:
import os
import sys
import subprocess
import numpy as np
import matplotlib.pyplot as plt

# If running on Colab, clone repo and install in editable mode
if "COLAB_GPU" in os.environ or "COLAB_RELEASE_TAG" in os.environ:
    if not os.path.exists("qig-code"):
        subprocess.run(["git", "clone", "https://github.com/lawrennd/qig-code.git"], check=True)
    os.chdir("qig-code")
    subprocess.run([sys.executable, "-m", "pip", "install", "-e", "."], check=True)
    # Ensure this notebook path is reachable
    sys.path.append(os.getcwd())

# Plot style
plt.rcParams.update({
    "figure.figsize": (6, 4),
    "axes.grid": True,
    "axes.spines.top": False,
    "axes.spines.right": False,
})

# Sanity check versions
import qig
print("qig version:", getattr(qig, "__version__", "dev"))


## Imports and helpers


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

from qig.exponential_family import QuantumExponentialFamily
from tests.tolerance_framework import QuantumTolerances

plt.rcParams.update({
    "figure.figsize": (6, 4),
    "axes.grid": True,
    "axes.spines.top": False,
    "axes.spines.right": False,
})


def accuracy_stats(drho_test: np.ndarray, drho_ref: np.ndarray) -> Dict[str, float]:
    diff = drho_test - drho_ref
    max_abs = np.max(np.abs(diff))
    fro = np.linalg.norm(diff, "fro")
    ref_fro = np.linalg.norm(drho_ref, "fro")
    rel = fro / ref_fro if ref_fro > 1e-14 else fro
    return {"max_abs": max_abs, "fro": fro, "rel": rel}


def time_call(fn, *args, **kwargs):
    import time
    start = time.perf_counter()
    out = fn(*args, **kwargs)
    return time.perf_counter() - start, out



## Single-qubit comparison (n=2, params=3)

Compute ∂ρ/∂θ₀ using four methods, compare to finite differences, and time them.


In [ ]:
# Build system
exp_fam = QuantumExponentialFamily(n_sites=1, d=2)
np.random.seed(0)
theta = 0.2 * np.random.randn(exp_fam.n_params)
a = 0  # differentiate w.r.t. first parameter

# Finite-difference reference
_eps = 1e-8
theta_plus = theta.copy(); theta_plus[a] += _eps
rho_plus = exp_fam.rho_from_theta(theta_plus)
theta_minus = theta.copy(); theta_minus[a] -= _eps
rho_minus = exp_fam.rho_from_theta(theta_minus)
drho_fd = (rho_plus - rho_minus) / (2 * _eps)

methods = [
    ("Quadrature", {"method": "duhamel", "n_points": 50}),
    ("Spectral", {"method": "duhamel_spectral"}),
    ("Block", {"method": "duhamel_block"}),
    ("SLD", {"method": "sld"}),
]

rows = []
for name, kwargs in methods:
    t, drho = time_call(exp_fam.rho_derivative, theta, a, **kwargs)
    stats = accuracy_stats(drho, drho_fd)
    rows.append((name, t, stats))

print(f"{'Method':<12} {'Time (ms)':>10} {'Max Abs Err':>15} {'Rel Err':>12}")
for name, t, stats in rows:
    print(f"{name:<12} {t*1e3:10.3f} {stats['max_abs']:15.3e} {stats['rel']:12.3e}")

# Bar plot of timing
labels = [r[0] for r in rows]
times_ms = [r[1]*1e3 for r in rows]
plt.bar(labels, times_ms, color=["#555555", "#4c78a8", "#59a14f", "#f28e2b"])
plt.ylabel("Time (ms)")
plt.title("Single qubit: timing by method")
plt.show()

# Bar plot of accuracy (max abs err)
max_err = [r[2]['max_abs'] for r in rows]
plt.bar(labels, max_err, color=["#555555", "#4c78a8", "#59a14f", "#f28e2b"])
plt.yscale('log')
plt.ylabel("Max absolute error (log)")
plt.title("Single qubit: accuracy vs finite difference")
plt.show()



## Single-qutrit comparison (n=3, params=8)

Repeat for a qutrit to see scaling effects.


In [ ]:
# Build system
exp_fam = QuantumExponentialFamily(n_sites=1, d=3)
np.random.seed(1)
theta = 0.05 * np.random.randn(exp_fam.n_params)
a = 0

# FD reference
_eps = 1e-8
theta_plus = theta.copy(); theta_plus[a] += _eps
rho_plus = exp_fam.rho_from_theta(theta_plus)
theta_minus = theta.copy(); theta_minus[a] -= _eps
rho_minus = exp_fam.rho_from_theta(theta_minus)
drho_fd = (rho_plus - rho_minus) / (2 * _eps)

methods = [
    ("Quadrature", {"method": "duhamel", "n_points": 50}),
    ("Spectral", {"method": "duhamel_spectral"}),
    ("Block", {"method": "duhamel_block"}),
    ("SLD", {"method": "sld"}),
]

rows = []
for name, kwargs in methods:
    t, drho = time_call(exp_fam.rho_derivative, theta, a, **kwargs)
    stats = accuracy_stats(drho, drho_fd)
    rows.append((name, t, stats))

print(f"{'Method':<12} {'Time (ms)':>10} {'Max Abs Err':>15} {'Rel Err':>12}")
for name, t, stats in rows:
    print(f"{name:<12} {t*1e3:10.3f} {stats['max_abs']:15.3e} {stats['rel']:12.3e}")



## Quick scaling snapshot

Compare timing for the three main methods across qubit and qutrit cases.


In [ ]:
def benchmark_sizes(cases):
    records = []
    for (n_sites, d) in cases:
        exp_fam = QuantumExponentialFamily(n_sites=n_sites, d=d)
        theta = 0.1 * np.random.randn(exp_fam.n_params)
        a = 0
        for label, method in [("Spectral", "duhamel_spectral"),
                              ("Block", "duhamel_block"),
                              ("SLD", "sld")]:
            t, _ = time_call(exp_fam.rho_derivative, theta, a, method=method)
            records.append({"label": label, "time": t*1e3, "n": d**n_sites})
    return records

cases = [(1,2), (1,3), (2,2)]
data = benchmark_sizes(cases)

# Plot
plt.figure()
for label in ["Spectral", "Block", "SLD"]:
    xs = [r["n"] for r in data if r["label"] == label]
    ys = [r["time"] for r in data if r["label"] == label]
    plt.plot(xs, ys, marker='o', label=label)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('System dimension (n = d^sites)')
plt.ylabel('Time (ms, log-log)')
plt.title('Scaling snapshot')
plt.legend()
plt.show()



## Takeaways

- **Spectral** and **Block** achieve machine-precision accuracy on small systems; block avoids eigendecomposition and can be more robust on ill-conditioned cases.
- **Quadrature** is slower and less accurate; keep for validation.
- **SLD** is fastest but approximate (~1e-3); use when speed matters more than precision.
- For larger systems, spectral is typically faster than block; for ill-conditioned problems, block can be preferable.

See `docs/duhamel_methods_comparison.md` for a detailed table and guidance.
